In [1]:
from gezi.common import *
sys.path.append('..')
from src.config import *
from src.preprocess import *
from src.eval import *
gezi.init_flags()

In [2]:
# https://www.kaggle.com/code/goldenlock/ai4code-base?scriptVersionId=101148262
# 9025

In [3]:
root = '../working/offline/6/0'
context_model_name = 'deberta-v3-small.flag-context2-aug.n_context-40.cls_loss_rate-0.1.eval.p13-4'
pairwise_model_name = 'all-mpnet-base-v2.flag-pairwise13-4.pooling_mask-attention_mask.grad_acc-4'
pmodel2_name = 'deberta-v3-small.flag-pairwise14-2-cat.eval'

In [4]:
xc = gezi.load(f'{root}/{context_model_name}/valid.pkl')
xp = gezi.load(f'{root}/{pairwise_model_name}/valid.pkl')
xp2 = gezi.load(f'{root}/{pmodel2_name}/valid.pkl')

In [5]:
gezi.sort_dict_byid_(xc, 'cid')
gezi.sort_dict_byid_(xp, 'cid')
gezi.sort_dict_byid_(xp2, 'cid')

In [6]:
ids = set(xc['id'])

In [7]:
df_gt = pd.read_csv(f'{FLAGS.root}/train_orders.csv')
df_gt = df_gt[df_gt.id.isin(ids)]
df_gt['cell_order'] = df_gt['cell_order'].apply(lambda x: x.split())
df_gt.head()

,id,cell_order
4,0002115f48f982,"[9ec225f0, 18281c6c, e3b6b115, 4a044c54, 365fe..."
11,00062ab8487156,"[dcad687f, a2e1fc80, 7d977ee8, 45a82a59, cbbc3..."
28,000efd285fb982,"[74a30f80, ee2c8e08, 5523374e, ae8f8fe8, 2138e..."
39,0012865b766949,"[f9cb50e9, 25f7db90, d804e819, 6593a545, fc5bb..."
42,001308991e0c5e,"[21147235, 6c01d0d2, 5bd28595, b8fd3a8c, a2501..."


In [8]:
calc_metric(xc, 'reg_pred')

0.9006251692558017

In [9]:
calc_metric(xc, 'pred')

0.8992880214723898

In [10]:
calc_metric(xc, 'cls_pred')

0.896173129530509

In [11]:
calc_metric(xp, 'cls_pred')

0.8966581181771137

In [12]:
x = xc.copy()
x['pred'] = xc['reg_pred'] * 0.5 + xp['cls_pred'] * 0.5
calc_metric(x)

0.9044248355992179

In [13]:
x = xc.copy()
x['pred'] = xc['cls_pred'] * 0.5 + xp['cls_pred'] * 0.5
calc_metric(x)

0.9018247872358709

In [14]:
x = xc.copy()
x['pred'] = xc['reg_pred'] * 0.5 + xp['pred'] * 0.5
calc_metric(x)

0.9036697563235938

In [15]:
def merge(x, y, prob):
  # return y
  # return x
  if prob > 0.9:
    return x * (1 - 0.0001) + y * 0.0001
  elif abs(y - x) < 0.1:
    return x * (1 - 0.0001) + y * 0.0001
  elif abs(y - x) < 0.2:
    return x * 0.95 * prob + y * (1 - 0.95 * prob)
  elif abs(y - x) < 0.3:
    return x * 0.85 * prob + y * (1 - 0.85 * prob)
  elif abs(y - x) < 0.4:
    return x * 0.5 * prob + y * (1 - 0.5 * prob)
  else:
    return y

In [16]:
x['pred'] = [merge(x, y, prob) for x, y, prob in zip(xp['pred'], xc['pred'], xp['max_prob'])]
calc_metric(x)

0.908585923202479

In [17]:
df_p = pd.DataFrame(xp)

In [18]:
df_c = pd.DataFrame(gezi.batch2list(xc))

In [19]:
df = df_c.merge(df_p[['cid', 'pred', 'cls_pred', 'max_prob', 'max_sim', 'probs', 'sims']], on='cid', suffixes=['_c', '_p'])

In [20]:
df_train = pd.read_feather('../working/train.fea')

In [21]:
df_train = df_train[df_train.id.isin(ids)]

In [22]:
df = df.merge(df_train[['cid', 'ancestor_id', 'n_words', 'source']], on='cid')

In [23]:
gezi.set_fold(df, 5, 'ancestor_id')

In [24]:
df['markdown_frac'] = df['n_markdown_cell'] / df['n_cell']
df['span'] = 1 / (df['n_code_cell'] + 1)
top2, top3, top4, top5 = [], [], [], []
top2_prob, top3_prob, top4_prob, top5_prob = [], [], [], []
top2_sim, top3_sim, top4_sim, top5_sim = [], [], [], []
for i in tqdm(range(len(df)), desc='top'):
  # cls_preds = df['cls_pred_ori'].values[i]
  n_code = df['n_code_cell'].values[i]
  probs = df['probs'].values[i]
  sims = df['sims'].values[i]
  idxes = (-probs).argsort()
  if len(idxes) > 1:
    top2.append((idxes[1] + 0.5) / (n_code + 1))
    top2_prob.append(probs[idxes[1]])
    top2_sim.append(sims[idxes[1]])
  else:
    top2.append(-1)
    top2_prob.append(-1)
    top2_sim.append(-1)
  if len(idxes) > 2:
    top3.append((idxes[2] + 0.5) / (n_code + 1))
    top3_prob.append(probs[idxes[2]])
    top3_sim.append(sims[idxes[2]])
  else:
    top3.append(-1)
    top3_prob.append(-1)
    top3_sim.append(-1)
  if len(idxes) > 3:
    top4.append((idxes[3] + 0.5) / (n_code + 1))
    top4_prob.append(probs[idxes[3]])
    top4_sim.append(sims[idxes[3]])
  else:
    top4.append(-1)
    top4_prob.append(-1)
    top4_sim.append(-1)
  if len(idxes) > 4:
    top5.append((idxes[4] + 0.5) / (n_code + 1))
    top5_prob.append(probs[idxes[4]])
    top5_sim.append(sims[idxes[4]])
  else:
    top5.append(-1)
    top5_prob.append(-1)
    top5_sim.append(-1)
ctop_prob, ctop2, ctop3, ctop4, ctop2_prob, ctop3_prob, ctop4_prob = [], [], [], [], [], [], []
for i in tqdm(range(len(df)), desc='ctop'):
  preds = df['cls_pred_ori'].values[i]
  probs = gezi.softmax(preds)
  idxes = (-probs).argsort()
  ctop_prob.append(probs[idxes[0]])
  ctop2.append((idxes[1] + 0.5) / FLAGS.num_classes)
  ctop2_prob.append(probs[idxes[1]])
  ctop3.append((idxes[2] + 0.5) / FLAGS.num_classes)
  ctop3_prob.append(probs[idxes[2]])
  ctop4.append((idxes[3] + 0.5) / FLAGS.num_classes)
  ctop4_prob.append(probs[idxes[3]])
# for i in range(FLAGS.num_classes):
#   df[f'cls_pred{i}'] = df['cls_pred_ori'].apply(lambda x: x[i])
df['top2'] = top2
df['top2_prob'] = top2_prob
df['top2_sim'] = top2_sim
df['top3'] = top3
df['top3_prob'] = top3_prob
df['top3_sim'] = top3_sim
df['top4'] = top4
df['top4_prob'] = top4_prob
df['top4_sim'] = top4_sim
df['top5'] = top5
df['top5_prob'] = top5_prob
df['top5_sim'] = top5_sim
df['ctop_prob'] = ctop_prob
df['ctop2'] = ctop2
df['ctop2_prob'] = ctop2_prob
df['ctop3'] = ctop3
df['ctop3_prob'] = ctop3_prob
df['ctop4'] = ctop4
df['ctop4_prob'] = ctop4_prob
df['pred_p2'] = xp2['pred']
p2_max_prob = []
p2_top2 = []
p2_top2_prob = []
for i in tqdm(range(len(df)), desc='p2'):
  code_idxes = xp2['code_idxes'][i]
  n_code = df['n_code_cell'].values[i]
  probs = gezi.softmax(xp2['cls_pred_ori'][i])
  idxes = (-probs).argsort()
  p2_top2.append((code_idxes[idxes[1]] + 0.5) / (n_code + 1))
  p2_max_prob.append(probs[idxes[0]])
  p2_top2_prob.append(probs[idxes[1]])
df['p2_max_prob'] = p2_max_prob
df['p2_top2'] = p2_top2
df['p2_top2_prob'] = p2_top2_prob
  
df['pred_diff0'] = abs(df['pred_c'] - df['pred_p'])
df['pred_diff1'] = abs(df['reg_pred'] - df['pred_p'])
df['pred_diff2'] = abs(df['cls_pred_c'] - df['pred_p'])
df['pred_diff3'] = abs(df['cls2_pred'] - df['pred_p'])
df['pred_diff4'] = abs(df['pred_c'] - df['top2'])
df['pred_diff5'] = abs(df['reg_pred'] - df['top2'])
df['pred_diff6'] = abs(df['cls_pred_c'] - df['top2'])
df['pred_diff7'] = abs(df['cls2_pred'] - df['top2'])
df['pred_diff8'] = abs(df['pred_p'] - df['top2'])
df['pred_diff9'] = abs(df['cls_pred_c'] - df['top3'])
df['pred_diff10'] = abs(df['pred_p'] - df['top3'])
df['pred_diff11'] = abs(df['pred_c'] - df['pred_p2'])
df['pred_diff12'] = abs(df['reg_pred'] - df['pred_p2'])
df['pred_diff13'] = abs(df['cls_pred_c'] - df['pred_p2'])
df['rule_pred'] = [merge(x, y, prob) for x, y, prob in tqdm(zip(df.pred_p.values, df.pred_c.values, df.max_prob.values), total=len(df), desc='rule')]

top:   0%|          | 0/424943 [00:00<?, ?it/s]

ctop:   0%|          | 0/424943 [00:00<?, ?it/s]

p2:   0%|          | 0/424943 [00:00<?, ?it/s]

rule:   0%|          | 0/424943 [00:00<?, ?it/s]

In [25]:
reg_cols =  [
          'n_code_cell',
          'n_markdown_cell',
          'n_cell',
          'cls_pred_c',
          'pred_c',
          'reg_pred',
          'cls2_pred',
          'pred_p',
          'cls_pred_p',
          'rule_pred',
          'pred_diff0',
          'pred_diff1',
          'pred_diff2',
          'pred_diff3',
          'pred_diff4',
          'pred_diff5',
          'pred_diff6',
          'pred_diff7',
          'pred_diff8',
          'pred_diff9',
          'pred_diff10',
          'pred_diff11',
          'pred_diff12',
          'pred_diff13',
          'max_sim',
          'max_prob',
          'markdown_frac',
          'span',
          'top2',
          'top2_prob',
          'top2_sim',
          'top3',
          'top3_prob',
          'top3_sim',
          'top4',
          'top4_prob',
          'top4_sim',
          'top5',
          'top5_prob',
          'top5_sim',
          'ctop_prob', 
          'ctop2', 
          'ctop3', 
          'ctop4', 
          'ctop2_prob', 
          'ctop3_prob', 
          'ctop4_prob',
          'pred_p2',
          'p2_max_prob',
          'p2_top2',
          'p2_top2_prob',
        ]
# for i in range(FLAGS.num_classes):
#   reg_cols += [f'cls_pred{i}']
cat_cols = [
          
            ]
label_col = 'rel_rank'
cols = reg_cols + cat_cols

In [26]:
from catboost import CatBoostRegressor
import lightgbm as lgb
xgb_params = {'learning_rate': 0.02,
              'reg_lambda': 7.960622217848342e-07, 
              'subsample': 0.7422597612762745,
              'max_depth': 10, 
              'early_stopping_rounds': 500,
              'n_estimators': 10000,
              'cat_features': [],
              'loss_function': 'MAE',
              }

xgb_params2 = {'learning_rate': 0.09827605967564293,'tree_method':'gpu_hist', 'gpu_id':0,
               'early_stopping_rounds': 50,
               'n_estimators': 10000, }

params = {
          'boosting': 'gbdt',
          'objective': 'regression_l1',
          'metric': {'l1'},
          'num_leaves': 32,
          'min_data_in_leaf': 300,
          # 'max_depth': 10,
          'learning_rate': 0.02,
          "feature_fraction": 0.8,
          "bagging_fraction": 0.75,
          'min_data_in_bin':100,
          "lambda_l1": 1e-7,
          'lambda_l2': 1e-7,
          "random_state": 1024,
          "num_threads": 12,
          }

In [27]:
FOLDS = 5

In [31]:
merge_scores = []
cbt_scores, lgb_scores = [], []
scores = []
for fold in tqdm(range(FOLDS)):
  dvalid = df[df.fold==fold]
  dtrain = df[df.fold!=fold]
  X_train = dtrain[cols]
  y_train = dtrain[label_col]
  X_valid = dvalid[cols]
  y_valid = dvalid[label_col]
  cbt_model = CatBoostRegressor(**xgb_params)
  cbt_model.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
                verbose=500,
                )  
  cbt_model.save_model(f'../working/trees/{fold}.cbt')
  dvalid['cb_pred'] = cbt_model.predict(dvalid[cols])
  
  x = {'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values}
  x['pred'] = [merge(x, y, prob) for x, y, prob in zip(dvalid.pred_p.values, dvalid.pred_c.values, dvalid.max_prob.values)]
  merge_score = calc_metric(x, 'pred', df_gt)
  ic(abs(x['pred'] - dvalid['rel_rank']).mean(), merge_score)
  cbt_score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.cb_pred.values})
  ic(cbt_score)
  
  d_train = lgb.Dataset(X_train, y_train)
  d_valid = lgb.Dataset(X_valid, y_valid, reference=d_train)
  lgb_model = lgb.train(params,
                  d_train,
                  10000,
                  valid_sets=[d_train, d_valid],
                  verbose_eval=500,
                  early_stopping_rounds=500)
  lgb_model.save_model(f'../working/trees/{fold}.lgb')
  dvalid['lgb_pred'] = lgb_model.predict(dvalid[cols])
  
  lgb_score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.lgb_pred.values})
  ic(lgb_score)
  score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': (dvalid.cb_pred.values * 0.3 + dvalid.lgb_pred.values * 0.7)})
  ic(score)
  
  merge_scores.append(merge_score)
  cbt_scores.append(cbt_score)
  lgb_scores.append(lgb_score)
  scores.append(score)
  ic(fold, 
     np.asarray(merge_scores).mean(), 
     np.asarray(cbt_scores).mean(), 
     np.asarray(lgb_scores).mean(),
     np.asarray(scores).mean(),
     )
  

  0%|          | 0/5 [00:00<?, ?it/s]

0:	learn: 0.2544626	test: 0.2544626	test1: 0.2543977	best: 0.2543977 (0)	total: 67.1ms	remaining: 11m 11s
500:	learn: 0.0623257	test: 0.0623257	test1: 0.0650472	best: 0.0650472 (500)	total: 30.4s	remaining: 9m 35s
1000:	learn: 0.0600127	test: 0.0600127	test1: 0.0639836	best: 0.0639836 (1000)	total: 1m 11s	remaining: 10m 44s
1500:	learn: 0.0585698	test: 0.0585698	test1: 0.0636613	best: 0.0636613 (1500)	total: 1m 58s	remaining: 11m 11s
2000:	learn: 0.0575620	test: 0.0575620	test1: 0.0636038	best: 0.0636032 (1991)	total: 2m 39s	remaining: 10m 36s
2500:	learn: 0.0566180	test: 0.0566180	test1: 0.0636183	best: 0.0636008 (2026)	total: 3m 22s	remaining: 10m 8s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.06360083571
bestIteration = 2026

Shrink model to first 2027 iterations.


[07/19/22 17:56:25] 2749851879.py:22 in <module>
                    abs(x['pred'] - dvalid['rel_rank']).mean(): 0.06579407253591144
                    merge_score: 0.9070703368237418
[07/19/22 17:56:27] 2749851879.py:24 in <module>
                    cbt_score: 0.9110552050830005


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.033063 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12117
[LightGBM] [Info] Number of data points in the train set: 342411, number of used features: 51
[LightGBM] [Info] Start training from score 0.455056
Training until validation scores don't improve for 500 rounds
[500]	training's l1: 0.0641013	valid_1's l1: 0.065489
[1000]	training's l1: 0.0627585	valid_1's l1: 0.0644039
[1500]	training's l1: 0.0621628	valid_1's l1: 0.0640303
[2000]	training's l1: 0.0617093	valid_1's l1: 0.0637891
[2500]	training's l1: 0.0613294	valid_1's l1: 0.0636073
[3000]	training's l1: 0.0610143	valid_1's l1: 0.0634572
[3500]	training's l1: 0.0607024	valid_1's l1: 0.0633355
[4000]	training's l1: 0.0604758	valid_1's l1: 0.0632593
[4500]	training's l1: 0.0603	valid_1's l1: 0.0632132
[5000]	training's l1: 0.0601138	valid_1's l1: 0.0631522
[5500]	training's l1: 0.0599398	vali

[07/19/22 17:59:40] 2749851879.py:38 in <module>
                    lgb_score: 0.9114947248763512
[07/19/22 17:59:41] 2749851879.py:40 in <module>
                    score: 0.9117161527798324
[07/19/22 17:59:41] 2749851879.py:46 in <module>
                    fold: 0
                    np.asarray(merge_scores).mean(): 0.9070703368237418
                    np.asarray(cbt_scores).mean(): 0.9110552050830005
                    np.asarray(lgb_scores).mean(): 0.9114947248763512
                    np.asarray(scores).mean(): 0.9117161527798324


...

[07/19/22 19:28:40] 2749851879.py:38 in <module>
                    lgb_score: 0.9170191022445477
[07/19/22 19:28:43] 2749851879.py:40 in <module>
                    score: 0.91725629300348
[07/19/22 19:28:43] 2749851879.py:46 in <module>
                    fold: 3
                    np.asarray(merge_scores).mean(): 0.9095752070572207
                    np.asarray(cbt_scores).mean(): 0.913523251293458
                    np.asarray(lgb_scores).mean(): 0.9141131060224191
                    np.asarray(scores).mean(): 0.914298878931308


0:	learn: 0.2544319	test: 0.2544319	test1: 0.2545198	best: 0.2545198 (0)	total: 383ms	remaining: 1h 3m 51s
500:	learn: 0.0622288	test: 0.0622288	test1: 0.0650961	best: 0.0650961 (500)	total: 1m 39s	remaining: 31m 30s
1000:	learn: 0.0597644	test: 0.0597644	test1: 0.0640164	best: 0.0640164 (1000)	total: 2m 48s	remaining: 25m 10s
1500:	learn: 0.0583114	test: 0.0583114	test1: 0.0637223	best: 0.0637220 (1498)	total: 3m 45s	remaining: 21m 16s
2000:	learn: 0.0570179	test: 0.0570179	test1: 0.0635879	best: 0.0635874 (1999)	total: 4m 44s	remaining: 18m 57s
2500:	learn: 0.0558048	test: 0.0558048	test1: 0.0635335	best: 0.0635329 (2488)	total: 5m 57s	remaining: 17m 51s
3000:	learn: 0.0548574	test: 0.0548574	test1: 0.0635302	best: 0.0635249 (2856)	total: 7m 42s	remaining: 17m 59s
3500:	learn: 0.0540844	test: 0.0540844	test1: 0.0635533	best: 0.0635235 (3152)	total: 9m 24s	remaining: 17m 27s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.06352346061
bestIteration = 3152

Shrink m

[07/19/22 19:38:51] 2749851879.py:22 in <module>
                    abs(x['pred'] - dvalid['rel_rank']).mean(): 0.06573988537486983
                    merge_score: 0.9039693073200067
[07/19/22 19:38:54] 2749851879.py:24 in <module>
                    cbt_score: 0.9072678199781201


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.030733 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12115
[LightGBM] [Info] Number of data points in the train set: 341935, number of used features: 51
[LightGBM] [Info] Start training from score 0.456522
Training until validation scores don't improve for 500 rounds
[500]	training's l1: 0.0640428	valid_1's l1: 0.0656852
[1000]	training's l1: 0.062795	valid_1's l1: 0.0646732
[1500]	training's l1: 0.0621823	valid_1's l1: 0.0642807
[2000]	training's l1: 0.0617099	valid_1's l1: 0.0639999
[2500]	training's l1: 0.0613732	valid_1's l1: 0.063815
[3000]	training's l1: 0.061076	valid_1's l1: 0.0636712
[3500]	training's l1: 0.060834	valid_1's l1: 0.063567
[4000]	training's l1: 0.0606172	valid_1's l1: 0.0634763
[4500]	training's l1: 0.0604379	valid_1's l1: 0.0634081
[5000]	training's l1: 0.06027

[07/19/22 19:58:03] 2749851879.py:38 in <module>
                    lgb_score: 0.9077804888224067
[07/19/22 19:58:05] 2749851879.py:40 in <module>
                    score: 0.9079099152585337
[07/19/22 19:58:05] 2749851879.py:46 in <module>
                    fold: 4
                    np.asarray(merge_scores).mean(): 0.908454027109778
                    np.asarray(cbt_scores).mean(): 0.9122721650303905
                    np.asarray(lgb_scores).mean(): 0.9128465825824167
                    np.asarray(scores).mean(): 0.9130210861967532


In [32]:
gezi.plot.feature_importance(cbt_model, topn=20)

UnboundLocalError: local variable 'feat_importances' referenced before assignment

In [ ]:
gezi.plot.feature_importance(lgb_model, topn=20)